In [48]:
from book_cover import Book
import os
import requests
from IPython.display import Image
from typing import Optional
import json
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
import io
from PIL import Image as PILImage


import importlib

def reload_book_class():
    """
    Reload and return the latest definition of Book class from book_cover.py.
    Useful during development in notebooks.
    """
    import book_cover
    importlib.reload(book_cover)
    return book_cover.Book



In [49]:
reload_book_class()

book_cover.Book

In [50]:
load_dotenv(override=True)

True

In [51]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"


In [52]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [53]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [54]:
def record_unknown_question_about_book(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [55]:

def find_books(query)-> Optional[list[Book]]:
    books = Book.get_books(query, limit=3)
    
    if books:
        print(f"Found {len(books)} books for {query}")
        return books
    else:
        print(f"No books found for {query}")
        return None


In [56]:
def get_book_cover_image(isbn) -> Optional[Image]:
   result = Book.get_book_cover_image(isbn) 
   return result


In [57]:
find_books_json = {
    "name": "find_books",
    "description": "Always use this tool to find books if the user asks about books or authors (writers)",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query to search the books or writers"
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [58]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [59]:
record_unknown_question_about_book_json = {
    "name": "record_unknown_question_about_book",
    "description": "Always use this tool to record any question about book or author (writer) that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [60]:
tools = [{"type": "function", "function": find_books_json},
        {"type": "function", "function": record_unknown_question_about_book_json},
        {"type": "function", "function": record_user_details_json}]


In [61]:
tools 

[{'type': 'function',
  'function': {'name': 'find_books',
   'description': 'Always use this tool to find books if the user asks about books or authors (writers)',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'query to search the books or writers'}},
    'required': ['query'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question_about_book',
   'description': "Always use this tool to record any question about book or author (writer) that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being in touch and provided an 

In [62]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!
        result = None
        is_find_books = False
        if tool_name == "find_books":
            result = find_books(**arguments)
            is_find_books = True
        elif tool_name == "record_unknown_question_about_book":
            result = record_unknown_question_about_book(**arguments)
        elif tool_name == "record_user_details":
            result = record_user_details(**arguments)

        # Convert Book objects to dicts for JSON serialization
        if is_find_books and result:
            content = json.dumps([{"title": b.title, "author": b.author, "year": b.year, "isbns": b.isbns} for b in result])
        else:
            content = json.dumps(result)
            
        results.append({"role": "tool", "content": content, "tool_call_id": tool_call.id})

        books = result if is_find_books else None
    return results, books 

In [63]:
system_prompt = (
    "You are an expert on artists and famous paintings they have painted. "
    "You can answer questions about famous painters. "
    "But you can also be asked about books or writers who wrote some books. "
    "In case you are asked about books or writers that wrote some books, "
    "try to find the book first. You can use the find_books tool to get a list of books that this writer has written. "
    "If the find_books tool didn't find any books, use the record_unknown_question_about_book tool to record the question you didn't find. "
    "If the user is engaging in discussion, steer them towards getting in touch via email; ask for their email and record it using the record_user_details tool. "
    "With this context, please chat with the user, always staying in character as a specialist in painting."
)

# system_prompt = (

#     "You are a sophisticated Art Historian and a specialist in famous paintings. You are cultured, eloquent, and possess an encyclopedic knowledge of both visual arts and classic literature. \
#     Task: Chat with the user about painters and their works. If the conversation shifts to literature, assist them in finding books. Your ultimate goal is to provide expert insight while capturing the user's contact information for future professional discussion. \
#     Context: The user is interested in high culture. You must remain in character as an art expert even when discussing writers or books. \
#     Format: Sophisticated conversational prose. \
#     Constraints & Workflow: \
#     - If asked about books/writers, first use the `find_books` tool. \
#     - If no books are found, use the `record_unknown_question_about_book` tool. \
#     - If a discussion develops, steer the user toward professional contact via email. \
#     - Ask for their email and record it using the `record_user_details` tool. \
#     - Use 'Chain-of-Thought': Analyze if the user's query is about art or books before responding, and check if it is the right moment to ask for an email."

# )



In [64]:
openai = OpenAI()

def chat(history):
    # Gradio passes (user_question, ai_answer text); use only the user message (single turn).
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history

    done = False
    books = None
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results, books = handle_tool_calls(tool_calls)
            # print("handle_tool_calls results", results)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
            
    content = response.choices[0].message.content
    content = content if content is not None else "(No response)"



    new_history = history + [{"role": "assistant", "content": content}]
    book_image = None
    if books:
        isbn = Book.get_book_isbn(books)
        if isbn and isbn is not None:
            data = Book.get_book_cover_image(isbn)
            if data:
                book_image = PILImage.open(io.BytesIO(data))
        else:
            book_image_bytes = Book.get_book_cover_image_not_available()
            book_image = PILImage.open(io.BytesIO(book_image_bytes))
                
    return new_history, book_image

In [65]:
# Gradio app: row1 = AI answer (TextBox) + book cover (Image), row2 = user question (TextBox) + Submit button

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]


with gr.Blocks() as ui:
    with gr.Row():
        ai_answer = gr.Chatbot(height=500)
        book_image = gr.Image(label="book cover", height=500, interactive=False)
    with gr.Row():
        user_question = gr.Textbox(label="Ask a question about famous artists and their works or writers (authors) and their bestsellers", placeholder="Author Dan Brown")

    # Allow submit by clicking the button or pressing Enter in the user_question textbox
    user_question.submit(fn=put_message_in_chatbot, inputs=[user_question, ai_answer], outputs=[user_question, ai_answer]).then(
        chat, 
        inputs=ai_answer, 
        outputs=[ai_answer, book_image])



In [66]:
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool called: find_books
Freida Mcfadden Bestsellers — Freida McFadden (2024) ISBN: —
The Housemaid — Freida McFadden (2022) ISBN: —
The Housemaid Is Watching — Freida McFadden (2024) ISBN: —
Found 3 books for Freida McFadden
Tool called: find_books
Year of Wonders — Geraldine Brooks (2001) ISBN: —
Study Guide - Caleb’s Crossing by Geraldine Brooks — SuperSummary (2019) ISBN: —
Geraldine Brooks' Year of Wonders — Virginia Lee (2013) ISBN: —
Found 3 books for Geraldine Brooks
Tool called: find_books
Writers & Lovers — Lily King (2020) ISBN: —
Euphoria A Novel — Lily King (2014) ISBN: —
Study Guide — SuperSummary (2019) ISBN: —
Found 3 books for Lily King
Tool called: find_books
The Purpose-Driven Life — Rick Warren (1997) ISBN: —
Summary of Rick Warren's Rick Warren's Bible Study Methods — Irb Media (2022) ISBN: —
Rick Warren & the modern church — Armin Hammer (2007) ISBN: —
Found 3 books for Rick Warren
Tool called: find_books
Catching Fire — Suzanne Collins (2009) ISBN: 9788372788757, 